<a href="https://colab.research.google.com/github/debaghtk/finetuning-poc/blob/main/finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


[hugging face peft tutorial](https://colab.research.google.com/github/huggingface/notebooks/blob/main/peft_docs/en/tensorflow/clm-prompt-tuning.ipynb#scrollTo=buySkzBqU1zS)

In [ ]:
%pip install transformers datasets torch

download model tokenizer and dataset

In [ ]:
from transformers import AutoModelForCausalLM as AutoModel, AutoTokenizer
from datasets import load_dataset

model_name = "bigscience/bloomz-560m"

model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
dataset = load_dataset("ought/raft", "twitter_complaints")

doing inference like TDD

In [ ]:
inputs = tokenizer(
    "Tweet text: @nationalgridus I have no water and the bill is current and paid. Can you do something about it? Label:",
    padding=True,
    max_length=64,
    truncation=True,
    return_tensors="pt",
)

output = model.generate(**inputs)
print(tokenizer.batch_decode(output, skip_special_tokens=True))

preprocessing of the data
- add text_label key, values - complaint, no complaint

In [ ]:
classes = [k.replace("_", " ") for k in dataset["train"].features["Label"].names]
dataset = dataset.map(
    lambda x: {"text_label": f'{classes[x["Label"]]}'},
)
dataset = dataset.map(
    lambda x: {"model_input": f'Tweet text: {x["Tweet text"]}, Label:'},
)
print(dataset["train"][0])

tokenize data

In [ ]:
tokenized_dataset = dataset.map(
    lambda x: tokenizer(x["model_input"], padding=True, truncation=True, max_length=64),
    batched=True,
)
print(tokenized_dataset["test"][0])

split data into train, test, validation

In [ ]:
train_dataset = tokenized_dataset["train"]
test_dataset = tokenized_dataset["test"]

